In [1]:
import ROOT

# Open file
f = ROOT.TFile.Open("higgsCombine.Test.GenerateOnly.mH125.123456.root")
f.ls()

# Get toys TTree
toys = f.Get("toys")
assert toys, "TTree 'toys' not found"

# Check available branches
toys.Print()

# Name of the Asimov entry/tree
# Sometimes toy_asimov is a *separate TTree*,
# sometimes it's an entry with toy index = -1
toy_asimov = toys.Get("toy_asimov")

# If toy_asimov exists as a TTree:
toy_asimov.Print()

# h = ROOT.TH1D(
#     "h",
#     "Asimov toy;observable;entries",
#     80, 100, 180
# )

# toy_asimov.Draw("CMS_hmm_mass >> h")  # <-- change observable
# c = ROOT.TCanvas("c", "c", 800, 600)
# h.Draw("hist")
# c.SaveAs("asimov_tree.png")

# Get the observable
x = toy_asimov.get().find("CMS_hgg_mass")   # <-- change name if needed
assert x, "Observable not found in RooDataSet"

# Define binning
nbins = x.getBinning().numBins()
xmin, xmax = x.getMin(), x.getMax()
x.setBins(nbins)

# Create histogram from RooDataSet
hist = toy_asimov.createHistogram(
    "h_asimov",
    x,
    ROOT.RooFit.Binning(nbins)
)

hist.SetTitle("Asimov dataset;CMS_hmm_mass;Events")

# Draw
c = ROOT.TCanvas("c", "c", 800, 600)
hist.Draw("hist")
c.SaveAs("asimov_hist.png")



TFile**		higgsCombine.Test.GenerateOnly.mH125.123456.root	
 TFile*		higgsCombine.Test.GenerateOnly.mH125.123456.root	
  KEY: TDirectoryFile	toys;1	toys
  KEY: TProcessID	ProcessID0;1	5566ce4a-e201-11f0-8c60-3902050abeef
  KEY: TTree	limit;1	limit
RooDataSet::model_sData[CMS_hgg_mass,CMS_channel,weight:_weight_] = 100 entries (10181 weighted)


Info in <TCanvas::Print>: png file asimov_hist.png has been created


In [4]:
toy_asimov.Print()

RooDataSet::model_sData[CMS_hgg_mass,CMS_channel,weight:_weight_] = 100 entries (10181 weighted)


In [8]:
nbins

100

In [13]:




import ROOT
from ROOT import RooFit, RooRealVar, RooGaussian, RooDataHist

def extract_chi2_from_roofit():
    # Setup the model
    x = RooRealVar("x", "x", -10, 10)
    mean = RooRealVar("mean", "mean", 0, -10, 10)
    sigma = RooRealVar("sigma", "sigma", 3, 0.1, 10)
    gauss = RooGaussian("gauss", "gauss", x, mean, sigma)

    # Generate a binned dataset (RooDataHist)
    # Using a data histogram is necessary for a proper chi^2 calculation within RooFit
    datahist = gauss.generateBinned(x, 10000) 
    
    # Perform a chi^2 fit and save the RooFitResult
    # The 'Save(ROOT.kTRUE)' or 'Save()' argument is crucial to return the result object
    fit_result = gauss.fitTo(datahist, RooFit.Save(ROOT.kTRUE), RooFit.PrintLevel(-1))
    
    # Extract the chi^2 value from the fit result
    # The minimized FCN value from a chi^2 fit is the chi^2 value itself
    chi2_value = fit_result.Chi2() # Alternative to fit_result.Chi2()
    # or using the Chi2() method (for binned likelihood fits, this returns 2*negative log-likelihood ratio, 
    # but for chi2FitTo it should be the chi2 value)
    # chi2_value = fit_result.Chi2() 

    # Get the number of degrees of freedom (optional but useful)
    # n_floating_params = fit_result.floatParsFinal().getSize()
    # n_bins = datahist.numEntries() # This is number of entries, not bins. Need to get bins from the axis.
    # ndf = n_bins - n_floating_params 
    # A more robust way to get the NDF for binned data is often via a RooPlot method, but here we just focus on the value.

    print(f"The extracted chi^2 value is: {chi2_value:.4f}")
    
    # Optional: Print the full fit result for verification
    # fit_result.Print()

if __name__ == "__main__":
    extract_chi2_from_roofit()


AttributeError: 'RooFitResult' object has no attribute 'chi2'

[#1] INFO:Fitting -- RooAbsPdf::fitTo(gauss_over_gauss_Int[x]) fixing normalization set for coefficient determination to observables in data
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_gauss_over_gauss_Int[x]_genData) Summation contains a RooNLLVar, using its error level
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: activating const optimization
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: deactivating const optimization


In [108]:
import ROOT

# Observable
x = ROOT.RooRealVar("x", "x", -5, 5)

# Define binning explicitly (important!)
nbins=100
x.setBins(nbins)

# Gaussian PDF
mean  = ROOT.RooRealVar("mean",  "mean",  0, -5, 5)
sigma = ROOT.RooRealVar("sigma", "sigma", 1,  0.1, 5)
pdf   = ROOT.RooGaussian("pdf", "pdf", x, mean, sigma)

# Generate unbinned data
data = pdf.generate(ROOT.RooArgSet(x), 10_000_000)

# Convert to binned dataset
datahist = ROOT.RooDataHist(
    "datahist",
    "datahist",
    ROOT.RooArgSet(x),
    data
)

# Fit to binned data
# pdf.fitTo(datahist, ROOT.RooFit.PrintLevel(-1))
fitresult = pdf.chi2FitTo(datahist, ROOT.RooFit.PrintLevel(-1), Save=True)

# # Chi2 calculation
# chi2 = ROOT.RooChi2Var(
#     "chi2",
#     "chi2",
#     pdf,
#     datahist,
#     ROOT.kTRUE,
#     0,
#     # ROOT.RooFit.DataError(ROOT.RooAbsData.Poisson)
# )

# # Extract values
# chi2_value = chi2.getVal()
# # ndf        = chi2.getNdf()

# print(f"Chi2      = {chi2_value}")
# nFit_params=2
# NDF = nbins-nFit_params
# print(f"NDF       = {NDF}")
# print(f"Chi2/NDF  = {chi2_value / NDF}")


[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: activating const optimization
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: deactivating const optimization


In [54]:
chi2.Print("v")

--- RooAbsArg ---
  Value State: clean
  Shape State: DIRTY
  Attributes: 
  Address: 0x5575258bd840
  Clients: 
  Servers: 
    (0x557525cd3e30,V-) RooRealVar::mean "mean"
    (0x557525cc3b60,V-) RooRealVar::sigma "sigma"
  Proxies: 
    parameters -> 
      1)   mean
      2)  sigma
--- RooAbsReal ---

  Plot label is "chi2"


In [103]:
fitresult.Print("v")


  RooFitResult: minimized FCN value: 19.2357, estimated distance to minimum: 0.000660891
                covariance matrix quality: Full, accurate covariance matrix
                Status : MINIMIZE=0 HESSE=0 

    Floating Parameter  InitialValue    FinalValue +/-  Error     GblCorr.
  --------------------  ------------  --------------------------  --------
                  mean    0.0000e+00    3.1470e-04 +/-  3.19e-04  <none>
                 sigma    1.0000e+00    1.0073e+00 +/-  2.25e-04  <none>



In [63]:
chi2_obj.Print("v")

--- RooAbsArg ---
  Value State: clean
  Shape State: DIRTY
  Attributes: 
  Address: 0x5575258bd840
  Clients: 
  Servers: 
    (0x557525cd3e30,V-) RooRealVar::mean "mean"
    (0x557525cc3b60,V-) RooRealVar::sigma "sigma"
  Proxies: 
    parameters -> 
      1)   mean
      2)  sigma
--- RooAbsReal ---

  Plot label is "chi2_pdf_datahist"


In [109]:
frame = x.frame()
datahist.plotOn(frame, Name="datahist")
pdf.plotOn(frame, Name="pdf")
chi2ndf = frame.chiSquare("pdf", "datahist", nFit_params)
print(f"chi2ndf      = {chi2ndf}")


chi2ndf      = 1.4091513401814881


In [113]:
# --- Chi2 via createChi2 ---
# This returns a RooAbsReal (actually a RooChi2Var under the hood for binned data)
chi2_obj = pdf.createChi2(
    datahist,
    # ROOT.RooFit.DataError(ROOT.RooAbsData.Poisson)
)

chi2 = chi2_obj.getVal()
chi2

113.88463929038639

In [112]:
c = ROOT.TCanvas("rf101_basics", "rf101_basics", 800, 400)
frame.Draw()
c.SaveAs("rf101_basics.png")

Warning in <TCanvas::Constructor>: Deleting canvas with same name: rf101_basics
Info in <TCanvas::Print>: png file rf101_basics.png has been created


In [117]:
chi2/(nbins-nFit_params)

1.162088156024351

In [116]:
chi2/chi2ndf

80.81789091278081

In [118]:
frame.chiSquare()

1.3809683133778583